In [6]:
import cv2
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from scipy.spatial import distance
import math

cv2.destroyAllWindows()

# Load YOLOv8 model
model = YOLO("yolov8n.pt")
video_path = "video_compressed.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Unable to open video file.")
    exit()

# Get video dimensions
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create a named window (no resizing, original size is maintained)
cv2.namedWindow("Vehicle Detection and Tracking", cv2.WINDOW_NORMAL)

# Mapping scale for real-world dimensions (calibration scale factor)
scale_factor = 0.05  # Adjust based on calibration (pixel to meter ratio)

def map_pixel_to_real_world(pixel_distance, scale_factor):
    return pixel_distance * scale_factor

# Preprocessing function (resizing, noise reduction, perspective transformation)
def preprocess_frame(frame):
    # Resize the frame if necessary (optional)
    frame_resized = cv2.resize(frame, (frame_width, frame_height))
    
    # Apply Gaussian Blur for noise reduction
    frame_blurred = cv2.GaussianBlur(frame_resized, (5, 5), 0)

    # Perspective Transformation (bird's-eye view)
    # Define source and destination points for perspective transformation
    src_pts = np.float32([[100, frame_height], [frame_width - 100, frame_height], [300, 200], [frame_width - 300, 200]])
    dst_pts = np.float32([[0, frame_height], [frame_width, frame_height], [0, 0], [frame_width, 0]])

    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    frame_perspective = cv2.warpPerspective(frame_blurred, matrix, (frame_width, frame_height))

    return frame_perspective

# Define regions (manually divide based on video structure)
road_regions = [
    {'name': 'Road 1', 'x1': 0, 'x2': int(frame_width / 5), 'y1': 0, 'y2': int(2 * frame_height / 4), 'color': (0, 255, 0), 'y_offset': 50},  # Left third
    {'name': 'Road 2', 'x1': int(frame_width / 5), 'x2': int(4 * frame_width / 5), 'y1': int(2 * (frame_height / 4)), 'y2': frame_height, 'color': (0, 0, 255), 'y_offset': 150},  # Middle third
    {'name': 'Road 3', 'x1': int(3 * frame_width / 5), 'x2': frame_width, 'y1': 0, 'y2': int(frame_height / 2), 'color': (255, 0, 0), 'y_offset': 250}  # Right third
]

# Vehicle dimensions (e.g., average width of cars and trucks in meters)
vehicle_widths = {'car': 2.0, 'truck': 2.5}  # Example values

frame_counter = 0  # Frame counter to log results frame by frame

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or unable to read frame.")
        break

    frame_counter += 1  # Increment frame counter

    # Preprocess the frame (resize, noise reduction, perspective transformation)
    processed_frame = preprocess_frame(frame)

    # Perform object detection
    results = model.predict(processed_frame)
    detections = []

    for result in results:
        for box in result.boxes.data:
            x1, y1, x2, y2, conf, cls = box.tolist()
            if int(cls) in [2, 7]:  # Detect cars and trucks (class 2 and 7)
                detections.append([int(x1), int(y1), int(x2), int(y2), conf])

    # Initialize list to store centroids and detected vehicles for each road
    road_widths = []

    # Process each road region
    for road in road_regions:
        # Extract the region of the frame for this road
        road_frame = processed_frame[road['y1']:road['y2'], road['x1']:road['x2']]

        centroids = []
        for det in detections:
            x1, y1, x2, y2, conf = det
            # Check if the detection is within the current road's x boundaries and y boundaries
            if road['x1'] <= x1 < road['x2'] and road['y1'] <= y1 < road['y2']:  
                cx = int((x1 + x2) / 2)
                cy = int((y1 + y2) / 2)
                centroids.append((cx, cy))
                # Draw rectangle around detected object
                cv2.rectangle(frame, (x1, y1), (x2, y2), road['color'], 2)
                # Draw centroid point
                cv2.circle(frame, (cx, cy), 5, (255, 255, 255), -1)

        # Calculate width for the road (the horizontal distance between centroids in the region)
        road_width = 0
        if len(centroids) > 1:
            farthest_distance = 0
            farthest_points = (None, None)
            for i, c1 in enumerate(centroids):
                for c2 in centroids[i + 1:]:
                    pixel_dist = distance.euclidean(c1, c2)
                    if pixel_dist > farthest_distance:
                        farthest_distance = pixel_dist
                        farthest_points = (c1, c2)

            # Calculate real-world distance from pixel distance
            real_distance = map_pixel_to_real_world(farthest_distance, scale_factor)
            road_width = real_distance

            if farthest_points[0] and farthest_points[1]:
                cv2.line(frame, farthest_points[0], farthest_points[1], road['color'], 2)
                # Display width on the frame
                cv2.putText(frame, f"{road['name']} Width: {road_width:.2f}m", 
                            (1, road['y_offset']), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Highlight the road region in the specified color
        cv2.rectangle(frame, (road['x1'], road['y1']), (road['x2'], road['y2']), road['color'], 2)

        # Store the road width for each frame
        road_widths.append((road['name'], road_width))

    # Save the frame results to the text file
    with open("road_width_results.txt", "a") as f:
        f.write(f"Frame {frame_counter}:\n")
        for road_name, width in road_widths:
            f.write(f"  {road_name}: {width:.2f} meters\n")
        f.write("\n")

    # Display the video frame without resizing (maintaining original size)
    cv2.imshow("Vehicle Detection and Tracking", frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 7 cars, 166.9ms
Speed: 5.0ms preprocess, 166.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 186.0ms
Speed: 4.6ms preprocess, 186.0ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 141.2ms
Speed: 5.3ms preprocess, 141.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 171.7ms
Speed: 6.0ms preprocess, 171.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 105.2ms
Speed: 4.0ms preprocess, 105.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 115.2ms
Speed: 4.7ms preprocess, 115.2ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 105.9ms
Speed: 3.0ms preprocess, 105.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 99.9ms
Speed: 4.1ms preprocess, 99.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384,

In [3]:
cv2.destroyAllWindows()